In [ ]:
import numpy as np
import pandas as pd
import os

import nimblephysics as nimble
from COMETH import Skeleton, DynamicSkeleton
gui = nimble.NimbleGUI()
gui.serve(8080)

# Draw the axes
axis_length=0.2
gui.nativeAPI().createLine(key='x_axis',points=[np.array([0,0,0]),np.array([axis_length,0,0])],color=np.array([255,0,0,1]))
gui.nativeAPI().createLine(key='y_axis',points=[np.array([0,0,0]),np.array([0,axis_length,0])],color=np.array([0,255,0,1]))
gui.nativeAPI().createLine(key='z_axis',points=[np.array([0,0,0]),np.array([0,0,axis_length])],color=np.array([0,0,255,1]))


GUIWebsocketServer will start serving a WebSocket server on ws://localhost:8070
Web GUI serving on http://localhost:8080


## BioAmass
The IK seems to be very stiff. Not good for training purposes.

In [ ]:
# Visualize the motion (go to http://localhost:8080, making sure in vs code you have ports 8080 and 8070 forwarded)
import time

skeleton = DynamicSkeleton()
# motion = nimble.biomechanics.OpenSimParser.loadMot(skeleton._nimble,'/home/emartini/nas/MAEVE/dataset/BioAmass/bioamass_v1.0/CMU/02/ab_fits/IK/02_ik.mot')
# motion = nimble.biomechanics.OpenSimParser.loadMot(skeleton._nimble,'/home/emartini/nas/MAEVE/dataset/BioAmass/bioamass_v1.0/DFAUST/50002/ab_fits/IK/shake_shoulders_ik.mot')
motion = nimble.biomechanics.OpenSimParser.loadMot(skeleton._nimble,'/home/emartini/nas/MAEVE/dataset/BioAmass/bioamass_v1.0/MPI_Limits/03099/ab_fits/IK/lar1_ik.mot')
gui.nativeAPI().renderSkeleton(skeleton._nimble,layer='1') 

for i in range(motion.poses.shape[1]-1):
    q = motion.poses[:,i]
    skeleton._nimble.setPositions(q)
    gui.nativeAPI().renderSkeleton(skeleton._nimble,layer='1') 
    time.sleep(motion.timestamps[i+1]-motion.timestamps[i])

# Some joints are locked (9 is the knee)
print(motion.poses[9,:])

## Human 3.6M

In [ ]:
# DEBUG: Visualize Example
markers = pd.read_csv('/home/emartini/nas/MAEVE/HUMAN_MODEL/dataset/csv_clean/S11_Directions.csv')

markers_dict = {
        'RKnee': 'RKnee',
        'LWrist': 'LWrist',
        'RHip': 'RHip',
        'RShoulder': 'RShoulder',
        'LElbow': 'LElbow',
        'LHip': 'LHip',
        'RElbow': 'RElbow',
        'RWrist': 'RWrist',
        'LKnee': 'LKnee',
        'LShoulder': 'LShoulder',
        'RAnkle': 'RAnkle',
        'LAnkle': 'LAnkle'
    }

Rz = np.array([
        [1,0,0],
        [0,1,0],
        [0,0,1]
    ])

Rx = np.array([
    [1,0,0 ],
    [0,0,-1 ],
    [0,1,0]
    ])

# Build the target 3D keypoints
target = []
for i in range(markers.shape[0]):
    row = []
    for kp in markers_dict.keys():
        p = np.array([markers[kp+":X"][i],markers[kp+":Y"][i],markers[kp+":Z"][i]])
        # rotate the 3d point -90 on the x axis (from y up to z up)
        # p_n = Rz.dot(p)
        p_n = p
        # p_n = Rz.dot(Rx.dot(p))
        row += p_n.tolist()
    target.append(np.array(row).reshape(-1,3))
target = np.array(target)

Setting len to 0.1
Setting neutral pos to   0
0.7
  0
Setting len to 0.3
Setting neutral pos to    0
-0.6
   0
Setting len to 0.1
Setting neutral pos to   0
0.2
  0


127.0.0.1 - - [13/Feb/2026 12:22:41] code 404, message File not found
127.0.0.1 - - [13/Feb/2026 12:22:41] "GET /v1/health HTTP/1.1" 404 -


In [ ]:
# DEBUG: Move the body model using gt to the first position (Check if it's all right)
s12 = Skeleton()
s = DynamicSkeleton()
s.hip_correction = True # From vicon we do not need correction
s.reset()
s._nimble.setPositions(np.zeros(49)) # Sometimes we nee to set the position to zero and not to normal (idk why)
s12.load_from_numpy(target[0].reshape(-1,3),s.kps)
s.load_from_BODY12(s12)
s.exact_scale(max_iterations=1000,precision=0.00001)
# s.qpIK(targets=[target[:].reshape(-1,3)])
gui.nativeAPI().renderSkeleton(s._nimble,layer='1') 

for i in range(len(markers_dict.keys())):
    gui.nativeAPI().createSphere(key=f'marker{i}', radii=np.array([0.01,0.01,0.01]),pos=target[0,i,:],color=np.array([0,0,255,1]))

# Move the skeleton through the entire sequence

# q = []
for t in range(target.shape[0]):
# for t in range(1000):
    s12.load_from_numpy(target[t].reshape(-1,3),s.kps)
    s.load_from_BODY12(s12)
    s.exact_scale(to_scale=True,max_iterations=100)

    # Save the angular position
    # q.append(s._nimble.getPositions())

    # Only for rendering purposes
    gui.nativeAPI().renderSkeleton(s._nimble,layer='1') 
    for i in range(len(markers_dict.keys())):
        gui.nativeAPI().createSphere(key=f'marker{i}', radii=np.array([0.01,0.01,0.01]),pos=target[t,i,:],color=np.array([0,0,255,1]))

127.0.0.1 - - [13/Feb/2026 12:24:11] code 404, message File not found
127.0.0.1 - - [13/Feb/2026 12:24:11] "GET /v1/health HTTP/1.1" 404 -


In [ ]:
# Record the IK on all the actions 
import os, glob
dir_in = '/home/emartini/nas/MAEVE/HUMAN_MODEL/dataset/csv_clean'
path_out = '/home/emartini/nas/MAEVE/HUMAN_MODEL/dataset_COMETH/H36M'

filenames = [os.path.basename(file) for file in glob.glob(os.path.join(dir_in,'*.csv'))]

s12 = Skeleton()
s = DynamicSkeleton()
s.hip_correction = True # From vicon we do not need correction

markers_dict = {
        'RKnee': 'RKnee',
        'LWrist': 'LWrist',
        'RHip': 'RHip',
        'RShoulder': 'RShoulder',
        'LElbow': 'LElbow',
        'LHip': 'LHip',
        'RElbow': 'RElbow',
        'RWrist': 'RWrist',
        'LKnee': 'LKnee',
        'LShoulder': 'LShoulder',
        'RAnkle': 'RAnkle',
        'LAnkle': 'LAnkle'
    }

for filename in filenames:
    markers = pd.read_csv(os.path.join(dir_in, filename))

    # Build the target array
    target = []
    for i in range(markers.shape[0]):
        row = []
        for kp in markers_dict.keys():
            p = np.array([markers[kp+":X"][i],markers[kp+":Y"][i],markers[kp+":Z"][i]])
            p_n = p
            row += p_n.tolist()
        target.append(np.array(row).reshape(-1,3))
    target = np.array(target)

    # IK
    s.reset()
    s._nimble.setPositions(np.zeros(49)) # Sometimes we nee to set the position to zero and not to normal (idk why)
    s12.load_from_numpy(target[0].reshape(-1,3),s.kps)
    s.load_from_BODY12(s12)
    s.exact_scale(max_iterations=1000,precision=0.00001)
    q = []
    for t in range(target.shape[0]):
        s12.load_from_numpy(target[t].reshape(-1,3),s.kps)
        s.load_from_BODY12(s12)
        s.exact_scale(to_scale=False,max_iterations=100)
        # Save the angular position
        q.append(s._nimble.getPositions())
    data = np.hstack([markers["time"].to_numpy().reshape((-1,1)),np.array(q)])
    np.savetxt(os.path.join(path_out,filename),data,fmt='%f', delimiter=',')

127.0.0.1 - - [13/Feb/2026 12:48:11] code 404, message File not found
127.0.0.1 - - [13/Feb/2026 12:48:11] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [13/Feb/2026 12:48:41] code 404, message File not found
127.0.0.1 - - [13/Feb/2026 12:48:41] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [13/Feb/2026 12:49:11] code 404, message File not found
127.0.0.1 - - [13/Feb/2026 12:49:11] "GET /v1/health HTTP/1.1" 404 -


TypeError: _savetxt_dispatcher() missing 1 required positional argument: 'X'

In [31]:
np.savetxt(os.path.join(path_out,filename),data,fmt='%f', delimiter=',')

127.0.0.1 - - [13/Feb/2026 12:51:11] code 404, message File not found
127.0.0.1 - - [13/Feb/2026 12:51:11] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [13/Feb/2026 12:51:41] code 404, message File not found
127.0.0.1 - - [13/Feb/2026 12:51:41] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [13/Feb/2026 12:52:11] code 404, message File not found
127.0.0.1 - - [13/Feb/2026 12:52:11] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [13/Feb/2026 12:52:41] code 404, message File not found
127.0.0.1 - - [13/Feb/2026 12:52:41] "GET /v1/health HTTP/1.1" 404 -
